In [1]:
#from utils.intermediate_layer_extraction import return_layer_input
import deepmatcher as dm

In [2]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../models/itunes_amazon_hybrid.pth')

In [3]:
from deepmatcher.data import MatchingIterator
from torch.autograd import Variable

In [4]:
def _return_input(module,module_input,module_output):
    global current_layer_input
    current_layer_input = module_input


    
def _flat_list(l):
    flat_list = []
    for sublist in l:
        for item in sublist:
            flat_list.append(item)
    return flat_list


def return_layer_input(dataset_dir,dataset_name,batch_size,model,layer,device = 0):
    dataset = dm.data.process(path=dataset_dir,train=dataset_name+'.csv',left_prefix='ltable_',right_prefix='rtable_',cache=dataset_name+'.pth')
    dataset_tuple = dataset,
    splits = MatchingIterator.splits(dataset_tuple,batch_size=batch_size, device = device)
    layer_inputs = []
    hook = layer.register_forward_hook(_return_input)
    batches = []
    for batch in splits[0]:
        model.forward(batch)
        batches.append(batch)
        layer_inputs.append(current_layer_input)
    hook.remove()
    return layer_inputs,batches


In [9]:
train_songname_hybrid,t_batches = return_layer_input('../Structured/itunes-amazon','merged_train',128,
                                                     hybrid_model,hybrid_model.attr_summarizers.Song_Name
                                                     ,device=-1)

In [10]:
left_input = train_songname_hybrid[0][0]
right_input = train_songname_hybrid[0][1]
songname_summarization = hybrid_model.attr_summarizers.Song_Name.forward(left_input,right_input)

In [17]:
predictions = hybrid_model.forward(t_batches[0])

In [24]:
import torch.nn.functional as F
F.softmax(predictions,dim=1)

Variable containing:
 0.6092  0.3908
 0.8112  0.1888
 0.0554  0.9446
 0.8403  0.1597
 0.0296  0.9704
 0.7458  0.2542
 0.7683  0.2317
 0.3680  0.6320
 0.6382  0.3618
 0.4029  0.5971
 0.0475  0.9525
 0.6964  0.3036
 0.7733  0.2267
 0.6890  0.3110
 0.7464  0.2536
 0.8725  0.1275
 0.8505  0.1495
 0.7985  0.2015
 0.1314  0.8686
 0.7582  0.2418
 0.6370  0.3630
 0.8222  0.1778
 0.8178  0.1822
 0.5367  0.4633
 0.8597  0.1403
 0.7510  0.2490
 0.7642  0.2358
 0.6320  0.3680
 0.8711  0.1289
 0.6112  0.3888
 0.1589  0.8411
 0.0056  0.9944
 0.8557  0.1443
 0.3568  0.6432
 0.8286  0.1714
 0.6823  0.3177
 0.3017  0.6983
 0.7427  0.2573
 0.7811  0.2189
 0.3354  0.6646
 0.8015  0.1985
 0.7364  0.2636
 0.8341  0.1659
 0.7709  0.2291
 0.5565  0.4435
 0.3194  0.6806
 0.7795  0.2205
 0.7067  0.2933
 0.7510  0.2490
 0.8397  0.1603
 0.0306  0.9694
 0.8800  0.1200
 0.7472  0.2528
 0.7326  0.2674
 0.3836  0.6164
 0.3337  0.6663
 0.6685  0.3315
 0.8049  0.1951
 0.6030  0.3970
 0.7289  0.2711
 0.8057  0.1943
 0.